# LCLUC Random Forest

In [17]:
# Import packages
import os
import geopandas as gpd
import pandas as pd
import numpy as np
import rioxarray as rioxr
import xarray as xr
import rasterstats
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix


pd.set_option('display.max_columns', None)


In [3]:
# Read in data
wrk_dir = Path.cwd().parent # go up a directory with parent
data_path = os.path.join(wrk_dir, 'data')

all_label_data = pd.read_parquet(os.path.join(data_path, 'all_label_data.parquet'))

In [4]:
# Create data set with only forest and non-forest labels 
def subset_labels(df, classes):
    """
    Create dataset with specified classes, combining all others as 'nonforest'
    """
    # Create a copy to avoid modifying original
    subset = df.copy()
    
    # Replace class values: keep if in classes list, otherwise set to 'nonforest'
    subset['class'] = subset['class'].apply(
        lambda x: x if x in classes else 'nonforest'
    )
    
    # Convert class column to categorical
    subset['class'] = subset['class'].astype('category')
    
    return subset

forest = subset_labels(all_label_data, ['forest'])
forest_water_bare = subset_labels(all_label_data, ['forest', 'water', 'bare'])

In [5]:
print(f"{forest['class'].value_counts()}\n")
print(forest_water_bare['class'].value_counts())

class
nonforest    1030
forest        270
Name: count, dtype: int64

class
nonforest    670
bare         270
forest       270
water         90
Name: count, dtype: int64


For the forest/non-forest, definitely have a class imbalance that should be considered in the model design...

In [8]:
forest.isna().sum()

class                      0
coastal_areosol_L30       19
blue_L30                  19
green_L30                 19
red_L30                   19
NIR_narrow_L30            19
SWIR1_L30                 19
SWIR2_L30                 19
cirrus_L30                19
TIR1_L30                  19
TIR2_L30                  19
coastal_areosol_S30        0
blue_S30                   0
green_S30                  0
red_S30                    0
red_edge_1_S30             0
red_edge_2_S30             0
red_edge_3_S30             0
NIR_broad_S30              0
NIR_narrow_S30             0
SWIR1_S30                  0
SWIR2_S30                  0
water_vapor_S30            0
cirrus_S30                 0
blue_evi_l30              19
blue_evi_s30               0
VV                         0
VH                         0
VV_VH                      0
dem_band_0                 0
dem_band_1                 0
dem_band_2                 0
dem_band_3                 0
dem_band_4                 0
dem_band_5    

## Model Design

In [ ]:
# Impute missing values
forest_clean = forest.drop(columns=['GLCM_correlation_nir', 'GLCM_contrast_red'])

# Separate features and target
X = forest_clean.drop(columns=['class'])
y = forest_clean['class']

# Scale to avoid warnings when imputing
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns, index=X.index)

# Impute remaining NAs using KNN
imputer = KNNImputer(n_neighbors=5)
X_imputed = pd.DataFrame(imputer.fit_transform(X_scaled), columns=X.columns)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.3, random_state=42, stratify=y)  # Ensures both train/test have same class ratios


Here is where you can add more parameters and vary the ranges of the parameters. This is just a few to get started, but there are many more you can add and also other ways of optimizing a parameter grid beyond a grid search

In [ ]:
# Parameter search
param_grid = {
    'n_estimators': [100, 300, 500],
    'max_depth': [10, 20, 30],
    'min_samples_split': [5, 10, 20],
    'class_weight': ['balanced', 'balanced_subsample']
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42, n_jobs=-1),
    param_grid,
    cv=5,
    scoring='f1_weighted',
    # verbose=2
)

grid_search.fit(X_train, y_train)

In [ ]:
# Feature Importance 
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': best_model.feature_importances_
}).sort_values('importance', ascending=False)

print(feature_importance.head(10))

                feature  importance
10  coastal_areosol_S30    0.167140
24         blue_evi_s30    0.126484
11             blue_S30    0.098665
5             SWIR1_L30    0.067222
2             green_L30    0.055974
1              blue_L30    0.044298
14       red_edge_1_S30    0.044013
6             SWIR2_L30    0.036907
20            SWIR2_S30    0.032166
12            green_S30    0.031920


In [ ]:
# See what the best parameter selections were
grid_search.best_params_

{'class_weight': 'balanced',
 'max_depth': 20,
 'min_samples_split': 5,
 'n_estimators': 100}

In [19]:
# Initiate model with best parameters
best_rf_model = RandomForestClassifier(
    **grid_search.best_params_,
    n_jobs=-1,
    random_state = 808
)

In [20]:
# Fit model
best_rf_model.fit(X_train, y_train)

y_pred = best_rf_model.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

      forest       0.99      0.98      0.98        81
   nonforest       0.99      1.00      1.00       309

    accuracy                           0.99       390
   macro avg       0.99      0.99      0.99       390
weighted avg       0.99      0.99      0.99       390

[[ 79   2]
 [  1 308]]


In [ ]:

# 6. EFFICIENT Raster Prediction (avoids memory issues)
# Stack rasters into numpy array
raster_stack = rasters.values  # shape: (bands, y, x)
n_bands, height, width = raster_stack.shape

# Reshape to (n_pixels, n_bands)
raster_2d = raster_stack.reshape(n_bands, -1).T


In [ ]:

# Convert to DataFrame with proper column names
raster_df = pd.DataFrame(raster_2d, columns=rasters.band.values)

# Apply same scaling transformation used during training
raster_scaled = pd.DataFrame(
    scaler.transform(raster_df),
    columns=raster_df.columns
)

# Impute NAs in raster data
raster_imputed = pd.DataFrame(
    imputer.transform(raster_df), 
    columns=raster_df.columns
)


In [ ]:

# 7. Predict in Chunks (memory efficient for large rasters)
chunk_size = 100000
predictions = []

for i in range(0, len(raster_imputed), chunk_size):
    chunk = raster_imputed.iloc[i:i+chunk_size]
    pred_chunk = rf_model.predict(chunk)
    predictions.extend(pred_chunk)

predictions = np.array(predictions)


In [ ]:

# 8. Recode to Numeric
label_map = {'forest': 1, 'nonforest': 0}
predictions_numeric = np.array([label_map.get(p, -1) for p in predictions])

# 9. Reshape back to raster dimensions
prediction_raster = predictions_numeric.reshape(height, width)

# 10. Save as GeoTIFF
output = rasters.isel(band=0).copy()  # Template from first band
output.values = prediction_raster
output.rio.to_raster(os.path.join(data_path, 'forest_prediction.tif'))